This notebook implements a GMM for the regression task.

In [ ]:
# Standard Python Imports
import os
import matplotlib.pyplot as plt

# Extra non-standard utilities
from argparse import Namespace


# Data management and Math imports
import numpy as np
import pandas as pd

# Torch Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset 
import copy

from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import mean_squared_error

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.πspeline import πspeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.linear_model import LinearRegression


In [2]:
args = Namespace(
   seed=5059 
) 

In [3]:
rng = np.random.default_rng(args.seed)

# 1 Load in Data

In [4]:
dataDir = "NumpyData/"

In [5]:
X_train_processed = np.load(dataDir + 'X_train_prepared.npy')
X_test_processed = np.load(dataDir + 'X_test_prepared.npy')
X_val_processed = np.load(dataDir + 'X_val_prepared.npy')

y_train_processed = np.load(dataDir + 'y_train_prepared.npy')
y_test_processed = np.load(dataDir + 'y_test_prepared.npy')
y_val_processed = np.load(dataDir + 'y_val_prepared.npy')

y_train = np.load(dataDir + 'y_train.npy')
y_val = np.load(dataDir + 'y_val.npy')
y_test = np.load(dataDir + 'y_test.npy')

Combine train and validation data - no need for validation

In [6]:
X_train_processed =  np.vstack((X_train_processed, X_val_processed))
y_train_processed =  np.vstack((y_train_processed, y_val_processed))
y_train = np.concatenate((y_train, y_val))

# 2 First principles GMM

## 2.1 Expectation Step

Gaussian Log Likelihood
$$\begin{align}
    P(x|\sigma_k^2,\mu_k) = -\frac{n}{2}\ln(2\pi\sigma^{2}_k) - \frac{1}{2\sigma^{2}}\sum_{i=1}^n (x - \mu_k)^2
\end{align}$$
Hence we have responsibility assignment:
$$\begin{align}
    r_{ik} = \ln\pi_k - \frac{d}{2}\ln(2\pi) - \frac{1}{2}\ln|\Sigma_{k}| - \frac{1}{2}(x^{(i)} - \mu_k)^{T}\Sigma_{k}^{-1}(x^{(i)} - \mu_k) \\ - \ln \left \{ \sum_{k'}^{K} \pi_{k'} + \exp \left \{ -\frac{1}{2}(x^{(i)} - \mu_{k'})^{T}\Sigma_{k'}^{-1}(x^{(i)} - \mu_{k'}) \right \} \right \} 
\end{align}$$

In [ ]:

def e_step(X, πs, μs, Σs, eps=1e-14):

    n, d = X.shape
    K = len(πs)
    
    # Responsibility matrix
    R = np.zeros((n, K))
    
    for k in range(K):
        # Calculate the inverse of the covariance matrix and its determinant
        sigma_inv = np.linalg.inv(Σs[k])
        sigma_det = np.linalg.det(Σs[k])
        
        # Compute the log-likelihood for each data point and each cluster
        diff = X - μs[k]  # Shape (n, d)
        exponent = np.sum(diff @ sigma_inv * diff, axis=1)  # Shape (n,)
        
        # Log of the Gaussian component likelihood
        log_likelihood_k = -0.5 * (d * np.log(2 * np.pi) + np.log(sigma_det) + exponent)
        
        # Update responsibility matrix using the log of the likelihood
        R[:, k] = np.log(πs[k] + eps) + log_likelihood_k
    
    # Compute the log-sum-exp to normalize the responsibilities (numerically stable)
    log_R_norm = np.log(np.sum(np.exp(R), axis=1, keepdims=True) + eps)
    
    # Normalize the responsibilities so they sum to 1 across clusters for each data point
    R = np.exp(R - log_R_norm)  # This is the responsibility matrix
    
    # Compute the total log-likelihood of the data
    loglik = np.sum(log_R_norm)  # Sum of the log of the norm across all data points
    
    return R, loglik


## 2.2 Maximization Step

Update $\mu_{k}$, $\pi_{k}$, $\Sigma_{k}$ : 
$$\begin{align}
    \mu_{k} &= \frac{1}{\sum^{n}_{i=1} r_{ik}}\\
    \pi_{k} &= \frac{1}{n} \sum^{n}_{i=1} r_{ik}\\
    \Sigma_{k} &= \frac{1}{\sum^{n}_{i=1} r_{ik}} \sum^{n}_{i=1}r_{ik}(x^{(i)}-\mu_{k})(x^{(i)}-\mu_{k})^T
\end{align}$$


In [9]:
def m_step(X, R):
    n, d = X.shape  # n is number of data points, d is the dimensionality
    K = R.shape[1]  # K is the number of components (clusters)
    
    # Initialize parameters
    pi = np.zeros(K)
    mu = np.zeros((K, d))
    sigma = np.zeros((K, d, d))
    
    # Update the priors (weights) pi_k
    pi = np.sum(R, axis=0) / n
    
    # Update the means (mu_k) for each component
    for k in range(K):
        mu[k] = np.sum(R[:, k][:, np.newaxis] * X, axis=0) / np.sum(R[:, k])
    
    # Update the covariance matrices (Sigma_k) for each component
    for k in range(K):
        diff = X - mu[k]
        sigma[k] = np.dot((R[:, k][:, np.newaxis] * diff).T, diff) / np.sum(R[:, k])
    
    return pi, mu, sigma
